In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.resnet50 import ResNet50
from keras.layers import GlobalAveragePooling2D, Dropout, Dense
from keras.models import Model
from keras import optimizers
import keras.backend as K

Using TensorFlow backend.


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
import os

In [3]:
os.chdir("..")

In [4]:
CONFIG_FILEPATH = "./config/default.ini"

In [5]:
import configparser
config = configparser.ConfigParser()

In [6]:
config.read(CONFIG_FILEPATH)

['./config/default.ini']

In [7]:
# Read and split dataset
train_csv = config["dataset_information"]["training_csv"]
dataset_folder = config["dataset_information"]["dataset_folder"]
dataset = pd.read_csv(train_csv, dtype=str)

In [8]:
train_dev, test = train_test_split(dataset, test_size=0.1)
train, dev = train_test_split(train_dev, test_size=0.1)

In [9]:
img_height, img_width, depth = 256, 256, 3

color_mode = "grayscale" if depth == 1 else "rgb"

num_classes = 5

In [10]:
TRAINING_DATA_GENERATOR = ImageDataGenerator(rescale=1./255)
VALIDATION_DATA_GENERATOR = ImageDataGenerator(rescale=1./255)
TEST_DATA_GENERATOR = ImageDataGenerator(rescale=1./255)

In [11]:
TRAINING_DATA = TRAINING_DATA_GENERATOR.flow_from_dataframe(dataframe=train, directory=dataset_folder, x_col="Filename", y_col="Drscore", class_mode="categorical", target_size=(img_height, img_width), color_mode=color_mode)
VALIDATION_DATA = VALIDATION_DATA_GENERATOR.flow_from_dataframe(dataframe=dev, directory=dataset_folder, x_col="Filename", y_col="Drscore", class_mode="categorical", target_size=(img_height, img_width), color_mode=color_mode)
TEST_DATA = TEST_DATA_GENERATOR.flow_from_dataframe(dataframe=test, directory=dataset_folder, x_col="Filename", y_col="Drscore", class_mode="categorical", target_size=(img_height, img_width), color_mode=color_mode)

Found 11457 validated image filenames belonging to 5 classes.
Found 1273 validated image filenames belonging to 5 classes.
Found 1415 validated image filenames belonging to 5 classes.


In [12]:
base_model = ResNet50(weights= None, include_top=True, input_shape=(img_height,img_width, depth), classes=num_classes)

W0721 09:52:53.421310 4527932864 deprecation_wrapper.py:119] From /anaconda3/envs/sic/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0721 09:52:53.434782 4527932864 deprecation_wrapper.py:119] From /anaconda3/envs/sic/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0721 09:52:53.439279 4527932864 deprecation_wrapper.py:119] From /anaconda3/envs/sic/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4185: The name tf.truncated_normal is deprecated. Please use tf.random.truncated_normal instead.

W0721 09:52:53.461320 4527932864 deprecation_wrapper.py:119] From /anaconda3/envs/sic/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0721

In [13]:
base_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 262, 262, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 128, 128, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 128, 128, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [14]:
sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

In [15]:
base_model.compile(loss="categorical_crossentropy", optimizer=sgd, metrics=['accuracy'])

W0721 09:52:57.590552 4527932864 deprecation_wrapper.py:119] From /anaconda3/envs/sic/lib/python3.6/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [16]:
base_model.fit_generator(generator=TRAINING_DATA, steps_per_epoch=1,
        epochs=1,
        validation_data=VALIDATION_DATA,
        validation_steps=1)

W0721 09:52:57.721935 4527932864 deprecation.py:323] From /anaconda3/envs/sic/lib/python3.6/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/1
1/1 [==============================] - 34s 34s/step - loss: 1.4724 - acc: 0.4062 - val_loss: 5.5406 - val_acc: 0.6562


In [18]:
base_model.save("trained_model.hdf5")
base_model.save_weights("trained_weights.hdf5")

In [21]:
PREDICTIONS = base_model.predict_generator(generator=TEST_DATA,
                                      steps=1)

In [22]:
import numpy as np

In [23]:
Y_PREDICTIONS = np.argmax(PREDICTIONS, axis=1)